![alt text][logo]

[logo]:https://wri-public-data.s3.amazonaws.com/resourcewatch/18_LOGO_ResourceWatch/18_LOGO_ResourceWatch_PB.png "Logo Title Text 2"

In [0]:
#@title # **Set-up Tool** { display-mode: "form" }
#@markdown ###Push the play button on the left to set-up the tool for your analysis.
!pip install geojson -q
import pandas as pd
import requests
import urllib.parse
import warnings
import numpy as np
import json
import shapely.wkt
import codecs
import random
import urllib.parse
import warnings
import geojson
import sys
warnings.simplefilter(action='ignore', category=FutureWarning)
!pip install LMIPy -q
import LMIPy
print('Done installing modules!')
print('You can move on to the next cell.')

In [0]:
#@title #**Choose Raster Dataset**
#@markdown ###Please enter a dataset ID, then push the play button on the left.
#@markdown Here are some example raster dataset IDs:
#@markdown 
#@markdown *  Annual Fishing Activity: com030-Fishing-by-Year
#@markdown *  Country Fishing Activity: com030-Fishing-by-Country
#@markdown *  Annual Trawling Activity: com030-Annual-Trawling-Activity
#@markdown *  Popuation density: soc072-Population-Grid-250-m
#@markdown *  Chlorophyll-a: bio037-Chlorophyll-a-2


raster_dataset_id = None
raster_layer_id = None
raster_layer_name = None
raster_ds_name = None
def get_band_name(layer):
  layer_sld = layer['attributes']['layerConfig']['body']['sldValue']
  list_of_words = layer_sld.split()
  index = [i for i, x in enumerate(list_of_words) if "SourceChannelName" in x]
  if index:
    source_channel = list_of_words[index[0]]
    band_name=source_channel.split('>')[1].split('<')[0]
  else:
    band_name = 'undefined'
  return band_name
try:
  #Get raster dataset
  raster_dataset_id ="" #@param {type:"string"}
  raster_dataset_id=raster_dataset_id.strip()
  #Get name from raster dataset
  url = 'https://api.resourcewatch.org/dataset/'+raster_dataset_id+'?includes=layer,metadata'
  r = requests.get(url)
  ds_info = r.json()
  for ds in ds_info['data']['attributes']['metadata']:
    if ds['attributes']['application']=='rw':
      raster_ds_name = ds['attributes']['name']
  print('Analyzing dataset: {}'.format(raster_ds_name))
  #Get layers from raster dataset
  print('Available layers for '+raster_ds_name +' dataset:')
  layer_dict = dict()
  layer_band_dict = dict()
  layer_info_dict = dict()
  layers = r.json()['data']['attributes']['layer']
  for layer in layers:
    if layer['attributes']['application']==['rw']:
      layer_name = layer['attributes']['name']
      layer_id = layer['id']
      layer_band_name = get_band_name(layer)
      layer_dict[layer_id]=layer_name
      layer_info_dict[layer_id]=layer
      layer_band_dict[layer_id]=layer_band_name
      print('     '+layer_id + '  '+layer_name)
  raster_layer_id = "" #@param {type:"string"}
  if raster_layer_id=="":
    print("Please enter a layer ID from these options, and push play again.")
  else:
    try:
      raster_layer_id=raster_layer_id.strip()
      raster_layer_name = layer_dict[raster_layer_id]
      band_name = layer_band_dict[raster_layer_id]
      layer_info = layer_info_dict[raster_layer_id]
      print('Analyzing raster layer: {}'.format(raster_layer_name))
      print('You can move on to the next cell.')
    except:
      print('Invalid layer ID. Please try again.')
except:
  print("We couldn't find that dataset ID. Please try again.")

In [0]:
#@title ## **Choose the Dataset that will Define Your Boundary Area**
#@markdown You can type the word **national** in the 'shapefile_dataset_id' field if you would like to do a national-level analysis. You do not need to enter anything in the 'shapefile_layer_id' for a national-level analysis.
#@markdown 
#@markdown If you would like to use boundaries from a different vector dataset on Resource Watch, find the dataset that contains the boundary you want to use, navigate to its metadate page, and copy the dataset ID into the 'shapefile_dataset_id' field, below. You can use boundaries from any polygon (non-point) vector dataset on Resource Watch.
#@markdown 
#@markdown If you are interested in sub-national administrative boundaries, you may want to use the Subnational Political Boundaries dataset (http://bit.ly/2YZov2Y), which has the following dataset ID:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Politcial-Boundaries-GADM-adminitrative-level-1-1490086842541
#@markdown 
#@markdown Here are some additional example vector dataset IDs that you might want to use for your boundary area:
#@markdown *   World Database on Protected Areas (http://bit.ly/2Z5ucfE): de452a4c-a55c-464d-9037-8c3e9fe48365 
#@markdown *   Maritime Boundaries (http://bit.ly/2YYCm9L): bf5877eb-399a-4237-b510-b1d41049e3bc
#@markdown 
#@markdown Once you have entered your dataset ID, you can push the play button on the left to see a list of layers available for this dataset. Choose the layer that contains your boundary area, and enter the id in the 'shapefile_layer_id' field and push play again.
#@markdown 


#@markdown ###Please enter a dataset ID, then push the play button on the left.

shapefile_dataset_id = None
shapefile_layer_id = None
shapefile_layer_name = None
shapefile_ds_name = None
try:
  #Get vector dataset
  shapefile_dataset_id = "" #@param {type:"string"}
  shapefile_dataset_id = shapefile_dataset_id.strip()
  if shapefile_dataset_id == 'national':
    country_analysis = 'Y'
    print('Analyzing data at the national-level.')
    print('You can move on to the next cell.')  
  else:
    country_analysis = 'N'
    if shapefile_dataset_id == ('de452a4c-a55c-464d-9037-8c3e9fe48365' or 'World-Database-on-Protected-Areas-Global-1490086842549'):
      wdpa = 'Y'
    else:
      wdpa = 'N'
    #Get name from raster dataset
    url = 'https://api.resourcewatch.org/dataset/'+shapefile_dataset_id+'?includes=layer,metadata'
    r = requests.get(url)
    for ds in r.json()['data']['attributes']['metadata']:
      if ds['attributes']['application']=='rw':
        shapefile_ds_name = ds['attributes']['name']
    print('Analyzing dataset: {}'.format(shapefile_ds_name))
    #Get layers from shapefile dataset
    print('Available layers for '+shapefile_ds_name +' dataset:')
    layer_dict = dict()
    layers = r.json()['data']['attributes']['layer']
    for layer in layers:
      if layer['attributes']['application']==['rw']:
        analysis_layer_name = layer['attributes']['name']
        analysis_layer_id =  layer['id']
        layer_dict[analysis_layer_id]=analysis_layer_name
        print('     '+analysis_layer_name+' - Layer ID: '+analysis_layer_id)
        #print('       -Layer ID: '+analysis_layer_id)
    shapefile_layer_id = ""#@param {type:"string"}
    if shapefile_layer_id=="":
      print("Please enter a layer ID from these options, and push play again.")
    else:
      try:
        shapefile_layer_id=shapefile_layer_id.strip()
        shapefile_layer_name = layer_dict[shapefile_layer_id]
        print('Analyzing layer: {}'.format(shapefile_layer_name))
        print('You can move on to the next cell.')
      except:
        print("We couldn't find that layer ID. Please try again.")
except:
  print("We couldn't find that dataset ID. Please try again.")
  

In [0]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
#@markdown ## **Search for Your Boundary Area by ID**
#@markdown If you are doing a national-level analysis, enter the ISO3 code for the country of interest. You will easily find the ISO3 code by googling "ISO3 code for" followed by the name of the country. In Indonesia, for example, the ISO3 code is IDN.
#@markdown
#@markdown If you are doing analysis for another shapefile dataset on Resource Watch, first, open the dataset that contains your boundary area of interest on the Resource Watch map. Click on the boundary area that you want to analyze data within, copy the Area ID from the window that pops up, and paste it in the 'area_id' field below.
#@markdown ###Please idenitfy the boundary area where you would like to summarize data by entering its Area ID from the interaction in the area_id field. Then, push the play button on the left.
orig_geo_id=None

url_len = 2000 #5400
def simplification_factor_picker(best_simple, best_len, SIMPLE_I, best_simple_high, best_len_high, lowest_simple, highest_simple):
  SIMPLE_I+=1
  #first we will sample a range of simplificaiton factors
  if SIMPLE_I==1:
    simplification_factor=0.01
  elif SIMPLE_I==2:
    simplification_factor=0.05
  elif SIMPLE_I==3:
    simplification_factor=0.1
  elif SIMPLE_I==4:
    simplification_factor=0.5
  #next we will narrow down the simplification factor between the best two solutions
  else:
    if not best_simple:
      simplification_factor = highest_simple + 0.1
    elif not best_simple_high:
      simplification_factor = lowest_simple - 0.001
    else:
      low_int = int(best_simple_high*10000)
      high_int = int(best_simple*10000)
      #print('generating a random number between {} and {}'.format(low_int, high_int))
      simplification_factor = random.randint(low_int,high_int)/10000
  #print('trying {}'.format(simplification_factor))
  return simplification_factor, SIMPLE_I

def simplification_algorithm(current_simple, current_len, shapefile_dataset_id, table, cartodb_id):
  SIMPLE_I = 0
  best_len = 0
  best_len_high = np.inf
  best_simple = None
  best_simple_high = None
  lowest_simple = np.inf
  highest_simple = 0
  print('Simplifying geometry...', end="")
  while SIMPLE_I<20 or (best_len<(url_len-1000) and SIMPLE_I<40):
    if SIMPLE_I <20:
      if SIMPLE_I<19:
        print('{}%...'.format(SIMPLE_I*5), end="")
      else:
        print('{}%...'.format(SIMPLE_I*5))
    elif SIMPLE_I==20:
      print('This boundary area is very complicated...simplifying further...')
    current_simple, SIMPLE_I = simplification_factor_picker(best_simple, best_len, SIMPLE_I, best_simple_high, best_len_high, lowest_simple, highest_simple)
    try:
      geo_id, feature = return_simplified_geometries(shapefile_dataset_id, table, cartodb_id, current_simple)
    except:
      continue
    if current_simple < lowest_simple:
      lowest_simple = current_simple
      #print('lowest_simple: {}'.format(lowest_simple))
    if current_simple > highest_simple:
      highest_simple = current_simple
      #print('highest_simple: {}'.format(highest_simple))
    current_len = len(str(feature))
    #print('Round {}, simplification factor: {}, length: {}'.format(SIMPLE_I, current_simple, current_len))

    if current_len < url_len and current_len > best_len:
      #print('New best feature!')
      best_len = current_len
      best_simple = current_simple
      best_geo_id = geo_id
      best_feature = feature
    elif current_len > url_len and current_len < best_len_high:
      #print('New best feature (high)!')
      best_len_high = current_len
      best_simple_high = current_simple
  return best_geo_id, best_feature, best_simple, best_len

def get_table_name(shapefile_layer_id):
  url= 'https://api.resourcewatch.org/layer/{shapefile_layer_id}'.format(shapefile_layer_id=shapefile_layer_id)
  r = requests.get(url)
  layer_sql = r.json()['data']['attributes']['layerConfig']['body']['layers'][0]['options']['sql']  
  list_of_words = layer_sql.split()
  indices = [i for i, x in enumerate(list_of_words) if x == "FROM"]
  possible_tables = []
  for idx in indices:
    next_word = list_of_words[idx + 1]
    if "select" not in next_word.lower():
      possible_tables.append(next_word)
  if len(np.unique(possible_tables))==1:
    table = np.unique(possible_tables)[0]
    carto_account = r.json()['data']['attributes']['layerConfig']['account']
  else:
    print("Sorry, we are having trouble finding the shapefiles for this layer. Please report the dataset you are searching for to amelia.snyder@wri.org so that she can fix this!")
    table = ""
  return table, carto_account

def return_geometries(table, carto_account, cartodb_id):
  headers = {
    'Content-Type': 'application/json',
  }
  params_rw=json.dumps({
      "provider":{
          "type": "carto",
          "table": table,
          "user": carto_account,
          "filter": "cartodb_id={}".format(cartodb_id)
      }
  })
  r = requests.post('https://api.resourcewatch.org/v1/geostore/', data=params_rw, headers=headers)
  geo_id=r.json()['data']['id']
  feature = json.dumps(r.json()['data']['attributes']['geojson']['features'][0]['geometry'])
  return geo_id, feature

def return_simplified_geometries(shapefile_dataset_id, table, cartodb_id, simplification_factor):
  sql_statement = "SELECT ST_AsText(ST_Simplify(the_geom, {simpl})) as the_geom FROM {table} WHERE cartodb_id={cartodb_id}".format(table=table, cartodb_id=cartodb_id, simpl=simplification_factor)
  url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=shapefile_dataset_id, sql=urllib.parse.quote(sql_statement))
  r = requests.get(url)
  geom_wkt_str=r.json()['data'][0]['the_geom']
  wkt = shapely.wkt.loads(geom_wkt_str)
  g = LMIPy.Geometry(s=wkt)
  geo_id = g.id
  feature = geojson.Feature(geometry=wkt, properties={}).geometry
  return geo_id, feature

if shapefile_dataset_id!="" and shapefile_layer_id!="":
  area_id = ""#@param {type:"string"}
  area_id = area_id.strip()
  #area_name = ""#@param {type:"string"}
  #area_name = area_name.strip()
  if area_id == "":
    print('Please enter the area ID of the area you want to analyze.')
  else:
    if country_analysis == 'Y':
      area_name = area_id
      print('Searching for {area}...'.format(area=area_name))
      try:
        url = 'https://api.resourcewatch.org/v2/geostore/admin/{iso}'.format(iso=area_name)
        r=requests.get(url)
        feature = json.dumps(r.json()['data']['attributes']['geojson']['features'][0]['geometry'])
        geo_id=r.json()['data']['id']
        print('We found it!')
        orig_geo_id = geo_id
        feature_len = len(str(feature))
        simplification_factor = 0
        if feature_len>url_len:
          table = 'wri_countries_a'
          sql_statement = "SELECT * FROM {table} WHERE iso_a3='{iso}'".format(table=table, iso=area_name)
          ds_id = '20662342-dcdd-4a42-9f58-bcc80217de71'
          url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=ds_id, sql=urllib.parse.quote(sql_statement))
          r = requests.get(url)
          df=pd.DataFrame(r.json()['data'])
          cartodb_id = df.loc[0, 'cartodb_id']
          geo_id, feature, simplification_factor, feature_len = simplification_algorithm(simplification_factor, feature_len, ds_id, table, cartodb_id)
        #print('final simplification factor: {}, final url length: {}'.format(simplification_factor, feature_len))
        if len(str(feature))<=url_len:
          print('Your boundary area has been successfully processed.')
          print('You can move on to the next cell.')
      except:
        print("We couldn't find {area}. Please check your ISO3 code and start over.".format(area=area_name))
    else:
      table, carto_account = get_table_name(shapefile_layer_id)
      if area_id!="":
        try:
          int(area_id)
        except Exception as e:
          print("Please make sure you entered the boundary area's name in the area_name field (not the area_id field).")
          raise StopExecution
        cartodb_id=area_id
        print('Searching for area ID {area} in {shp_name}...'.format(area=area_id, shp_name=shapefile_layer_name))
      elif area_name!="":
        print('Searching for {area} in {shp_name}...'.format(area=area_name, shp_name=shapefile_layer_name))
        sql_statement = "SELECT * FROM {table} LIMIT 1".format(table=table)
        url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=shapefile_dataset_id, sql=urllib.parse.quote(sql_statement))
        r = requests.get(url)
        df_full=pd.DataFrame(r.json()['data'])
        cols = df_full.columns
        cols = cols.drop('the_geom').drop('the_geom_webmercator')
        sql_statement = "SELECT {cols} FROM {table}".format(cols=", ".join(cols), table=table)
        url= 'https://api.resourcewatch.org/query/{dataset_id}?sql={sql}'.format(dataset_id=shapefile_dataset_id, sql=urllib.parse.quote(sql_statement))
        r = requests.get(url)
        df_full=pd.DataFrame(r.json()['data'])
        try:
          area_rows = df_full[df_full.eq(area_name).any(1)]
          indices = area_rows.index.tolist()
          if len(indices)>1:
            print('Please choose the cartodb_id for the area that you are interested in from the table below:')
            display(area_rows)
            cartodb_id = int(input("Which cartodb_id would you like to use?"))
          else:
            index = area_rows.index.tolist()[0]
            cartodb_id = area_rows.loc[index, 'cartodb_id']
          print('We found it!')
        except:
          print("We couldn't find that area. Check the name of your region and try again.")
          raise StopExecution
      try:
        geo_id, feature = return_geometries(table, carto_account, cartodb_id)
        orig_geo_id = geo_id
        feature_len = len(str(feature))
        simplification_factor = 0
        if feature_len>url_len:
          geo_id, feature, simplification_factor, feature_len = simplification_algorithm(simplification_factor, feature_len, shapefile_dataset_id, table, cartodb_id)
        #print('final simplification factor: {}, final url length: {}'.format(simplification_factor, feature_len))
      except:
        print('This boundary area is too complex to pull from the API. Simplifying boundary area. This may take a few minutes...')
        feature_len=999999999
        simplification_factor = 0.05
        geo_id, feature, simplification_factor, feature_len = simplification_algorithm(simplification_factor, feature_len, shapefile_dataset_id, table, cartodb_id)
        #print('final simplification factor: {}, final url length: {}'.format(simplification_factor, feature_len))
      if len(str(feature))<=url_len:
        print('Your boundary area has been successfully processed.')
        print('You can move on to the next cell.')  

In [0]:
#@title # **Perform Analysis**
#@markdown ###Push the play button on the left to run your analysis and see the results.
layer_suffix=None
layer_prop=None
if 'output' in layer_info['attributes']['interactionConfig']:
  if 'suffix' in layer_info['attributes']['interactionConfig']['output'][0]:
    layer_suffix = layer_info['attributes']['interactionConfig']['output'][0]['suffix']
  if 'property' in layer_info['attributes']['interactionConfig']['output'][0]:
    layer_prop = layer_info['attributes']['interactionConfig']['output'][0]['property']
layer_description = layer_info['attributes']['description']

url = 'https://api.resourcewatch.org/layer/'+raster_layer_id
r = requests.get(url)
asset =r.json()['data']['attributes']['layerConfig']['assetId']
sql_query = 'SELECT ST_SUMMARYSTATS() from {gee_asset_id}'.format(gee_asset_id=asset)
params = {"sql": sql_query,
          "geostore": geo_id}
params_global = {"sql": sql_query}
url = 'https://api.resourcewatch.org/query/{dataset_id}'.format(dataset_id=raster_dataset_id)
print('Layer description: ' + layer_description)
if layer_prop:
  print('Property: ' +layer_prop)
if layer_suffix:
  print('Property suffix:' +layer_suffix)
print('Resolution: '+ds_info['data']['attributes']['metadata'][0]['attributes']['info']['spatial_resolution'])
print('')
print('Calculating raster statistics over area {}...'.format(area_id))

#pull area for boundary of interest
g = LMIPy.Geometry(id_hash=geo_id)    
area = g.attributes['areaHa']/100
print('The total area of this area is {area} square kilometers (km^2).'.format(area=area))
try:
  #get results for  your region
  r = requests.get(url, params=params)
  #print(r.url)
  results = r.json()['data'][0]['st_summarystats']
  print('')
  print('Summary statistics for area {}:'.format(area_id))
  if band_name == 'undefined':
    band_name = [key for key, value in results.items()][0]
  for key, value in results[band_name].items():
    print('   '+key +': ' +str(value))
    
  # get global results
  r = requests.get(url, params=params_global)
  results = r.json()['data'][0]['st_summarystats']
  #print(r.url)
  print('')
  #print('Summary statistics at the global level:')
  #if band_name == 'undefined':
  #  band_name = [key for key, value in results.items()][0]
  #for key, value in results[band_name].items():
  #  print('   '+key +': ' +str(value))
except:
  print(requests.get(url).json())
  status = requests.get(url).json()['errors'][0]['status']
  if status==500:
    print('We are having some trouble calculating raster statistics on the API. Please report this to amelia.snyder@wri.org and try again later.')
  else:
    print(requests.get(url).json()['errors'][0])
    


In [0]:
#@title # **Map Analysis**
#@markdown ###Push the play button on the left to see the area of analysis.
g = LMIPy.Geometry(id_hash=geo_id)    
lmipy_raster = LMIPy.Layer(id_hash=raster_layer_id)
#print(lmipy_raster.intersect(g))
lmipy_raster.map(geometry=g)